In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
import numpy as np
np.bool = np.bool_

In [3]:
#current notebook name
notebook_name = __session__.replace('.ipynb','')[__session__.rfind('/')+1:] 

In [4]:
# HDFS base paths
hdfs_lakehouse_base_path = 'hdfs://localhost:9000/lakehouse/'
hdfs_warehouse_base_path = 'hdfs://localhost:9000/warehouse'

In [5]:
import os
dependencies = ["org.apache.spark:spark-avro_2.12:3.5.0",
                "io.delta:delta-iceberg_2.12:3.0.0"]
os.environ['PYSPARK_SUBMIT_ARGS']= f"--packages {','.join(dependencies)} pyspark-shell"
os.environ['PYARROW_IGNORE_TIMEZONE'] = 'true'

In [6]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName(notebook_name)
    .config("spark.log.level","ERROR")
    .config("spark.sql.warehouse.dir",hdfs_warehouse_base_path)
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.executorEnv.PYTHONPATH", "/path/to/your/venv/lib/python3.11/site-packages") \

    .enableHiveSupport()
    .getOrCreate()
)

25/03/01 17:09:10 WARN Utils: Your hostname, osbdet resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s1)
25/03/01 17:09:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/osbdet/.ivy2/cache
The jars for the packages stored in: /home/osbdet/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-iceberg_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4984b617-a2d0-4ef4-a19f-8f336303a6d5;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.5.0 in central


:: loading settings :: url = jar:file:/home/osbdet/.jupyter_venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.tukaani#xz;1.9 in central
	found io.delta#delta-iceberg_2.12;3.0.0 in central
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.1.1 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.3 in central
	found org.checkerframework#checker-qual;3.19.0 in central
	found com.google.errorprone#error_prone_annotations;2.10.0 in central
:: resolution report :: resolve 189ms :: artifacts dl 4ms
	:: modules in use:
	com.github.ben-manes.caffeine#caffeine;2.9.3 from central in [default]
	com.google.errorprone#error_prone_annotations;2.10.0 from central in [default]
	io.delta#delta-iceberg_2.12;3.0.0 from central in [default]
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.spark#spark-avro_2.1

In [7]:
trump_tweet_with_sentiment_analysis = spark.read.format("delta").load(f"{hdfs_lakehouse_base_path}/gold/trump_btc/trump_tweet_with_sentiment_analysis").cache()
btc_price = spark.read.format("delta").load(f"{hdfs_lakehouse_base_path}/silver/trump_btc/BTC/").cache()


In [8]:
btc_price.orderBy("date").show(5)

+------------+----+----+----+-----+------+-------------------+
|   Timestamp|Open|High| Low|Close|Volume|               date|
+------------+----+----+----+-----+------+-------------------+
|1.32541206E9|4.58|4.58|4.58| 4.58|   0.0|2012-01-01 11:01:00|
|1.32541212E9|4.58|4.58|4.58| 4.58|   0.0|2012-01-01 11:02:00|
|1.32541218E9|4.58|4.58|4.58| 4.58|   0.0|2012-01-01 11:03:00|
|1.32541224E9|4.58|4.58|4.58| 4.58|   0.0|2012-01-01 11:04:00|
| 1.3254123E9|4.58|4.58|4.58| 4.58|   0.0|2012-01-01 11:05:00|
+------------+----+----+----+-----+------+-------------------+
only showing top 5 rows



In [9]:
trump_tweet_with_sentiment_analysis.orderBy("date").show(20)

+-------------------+---------+--------------------+---------+--------+--------------------+---------------+
|               date|favorites|                  id|isRetweet|retweets|                text|sentiment_score|
+-------------------+---------+--------------------+---------+--------+--------------------+---------------+
|               NULL|      NaN|                 NaN|     NULL|     NaN|                NULL|            0.0|
|               NULL|      NaN|                 NaN|     NULL|     NaN|                NULL|            0.0|
|2011-08-22 22:33:46|    193.0|1.057394600749383...|    false|   299.0|We don't need ano...|        -0.6908|
|2011-09-06 22:23:34|    105.0|1.111727130155458...|    false|   167.0|@BarackObama tril...|        -0.5563|
|2011-09-09 20:30:23|     23.0|1.122313901285130...|    false|    80.0|@BarackObama---th...|          -0.25|
|2011-09-20 15:17:36|     11.0|1.161389434178109...|    false|    38.0|"@BarackObama's m...|            0.0|
|2011-10-07 21:21:0

In [10]:
trump_tweet_with_sentiment_analysis.orderBy("date").show(20)

+-------------------+---------+--------------------+---------+--------+--------------------+---------------+
|               date|favorites|                  id|isRetweet|retweets|                text|sentiment_score|
+-------------------+---------+--------------------+---------+--------+--------------------+---------------+
|               NULL|      NaN|                 NaN|     NULL|     NaN|                NULL|            0.0|
|               NULL|      NaN|                 NaN|     NULL|     NaN|                NULL|            0.0|
|2011-08-22 22:33:46|    193.0|1.057394600749383...|    false|   299.0|We don't need ano...|        -0.6908|
|2011-09-06 22:23:34|    105.0|1.111727130155458...|    false|   167.0|@BarackObama tril...|        -0.5563|
|2011-09-09 20:30:23|     23.0|1.122313901285130...|    false|    80.0|@BarackObama---th...|          -0.25|
|2011-09-20 15:17:36|     11.0|1.161389434178109...|    false|    38.0|"@BarackObama's m...|            0.0|
|2011-10-07 21:21:0

In [11]:
from pyspark.sql.functions import date_format
#removing the seconds from both DF
trump_tweet_with_sentiment_analysis = trump_tweet_with_sentiment_analysis.withColumn("date", date_format("date", "yyyy-MM-dd HH:mm"))
btc_price = btc_price.withColumn("date", date_format("date", "yyyy-MM-dd HH:mm"))


In [12]:
trump_tweet_with_sentiment_analysis.show(5)
btc_price.show(5)

+----------------+---------+--------------------+---------+--------+--------------------+---------------+
|            date|favorites|                  id|isRetweet|retweets|                text|sentiment_score|
+----------------+---------+--------------------+---------+--------+--------------------+---------------+
|2024-10-31 17:16|   8601.0|1.134029284408450...|    false|  2282.0|"I would like to ...|         0.8652|
|2024-09-30 17:11|   8655.0|1.132271418381557...|    false|  2516.0|"I promised to Ma...|         0.9151|
|2024-09-24 17:02|      0.0|1.131931321423966...|     true|     0.0|"RT @realtrumpcoi...|            0.0|
|2024-09-22 00:22|   7797.0| 1.13177878498912E17|    false|  2029.0|"https://justthen...|            0.0|
|2024-09-21 23:31|   7113.0|1.131776764644532...|    false|  1909.0|"“Trump Credits S...|          0.886|
+----------------+---------+--------------------+---------+--------+--------------------+---------------+
only showing top 5 rows

+------------+-------

In [13]:

from pyspark.sql.functions import col
#filtering the tweets with sentiment +- 0.5
filtered_tweets = trump_tweet_with_sentiment_analysis.filter((trump_tweet_with_sentiment_analysis.sentiment_score > 0.5) | (trump_tweet_with_sentiment_analysis.sentiment_score < -0.5))
#filtering because we only got data from 2012
filtered_tweets = filtered_tweets.filter(col("date") >= "2012-01-01")

filtered_tweets.orderBy('date').show()

+----------------+---------+--------------------+---------+--------+--------------------+---------------+
|            date|favorites|                  id|isRetweet|retweets|                text|sentiment_score|
+----------------+---------+--------------------+---------+--------+--------------------+---------------+
|2018-12-17 14:27|  92663.0|1.074657278974939...|    false| 20421.0|It is incredible ...|         0.6893|
|2019-04-30 19:56|  47900.0|1.123285120864092...|    false| 11112.0|China is adding g...|         0.7655|
|2019-07-19 14:06|   6049.0|1.152187923908976...|    false|  1562.0|....State which I...|         0.8932|
|2019-07-19 14:16|  70874.0|1.152190440692756...|    false| 12980.0|....State which I...|         0.8932|
|2019-08-08 16:38|  85995.0|1.159473909827297...|    false| 17705.0|As your President...|         0.9273|
|2019-09-02 13:57|  15495.0|1.168493188056829...|    false|  4006.0|....Average hourl...|         0.7717|
|2019-09-16 13:47|  32931.0|1.173564172635914.

In [14]:
correlated_df = btc_price.join(filtered_tweets, "date", "left")


In [15]:
correlated_df.show(5)

+----------------+------------+-------+-------+-------+-------+-----------+---------+----+---------+--------+----+---------------+
|            date|   Timestamp|   Open|   High|    Low|  Close|     Volume|favorites|  id|isRetweet|retweets|text|sentiment_score|
+----------------+------------+-------+-------+-------+-------+-----------+---------+----+---------+--------+----+---------------+
|2017-07-11 01:18|1.49972868E9|2419.94| 2420.0|2418.95|2418.95| 4.07239513|     NULL|NULL|     NULL|    NULL|NULL|           NULL|
|2017-07-11 01:19|1.49972874E9|2421.99| 2422.0|2419.18|2419.18|15.83349066|     NULL|NULL|     NULL|    NULL|NULL|           NULL|
|2017-07-11 01:20| 1.4997288E9|2419.91|2421.96|2419.18|2421.95| 2.71926784|     NULL|NULL|     NULL|    NULL|NULL|           NULL|
|2017-07-11 01:21|1.49972886E9|2421.96| 2422.0|2419.29| 2422.0|42.68201165|     NULL|NULL|     NULL|    NULL|NULL|           NULL|
|2017-07-11 01:22|1.49972892E9|2420.33|2422.98|2414.79|2414.79|11.52859532|     NUL

In [16]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lead, col

# Define window ordered by date (timestamp of tweet)
windowSpec = Window.orderBy("date")

# Get BTC Close Price 1 minute after the tweet
correlated_df = correlated_df.withColumn("next_close", lead("Close").over(windowSpec))

# Compute absolute and percentage price change
correlated_df = correlated_df.withColumn("price_change", col("next_close") - col("Close"))
correlated_df = correlated_df.withColumn("price_change_pct", (col("price_change") / col("Close")) * 100)

# Select only necessary columns
correlated_df = correlated_df.select("date", "price_change", "price_change_pct")

# Show results
correlated_df.show(10, truncate=False)



+----------------+------------+----------------+
|date            |price_change|price_change_pct|
+----------------+------------+----------------+
|2012-01-01 11:01|0.0         |0.0             |
|2012-01-01 11:02|0.0         |0.0             |
|2012-01-01 11:03|0.0         |0.0             |
|2012-01-01 11:04|0.0         |0.0             |
|2012-01-01 11:05|0.0         |0.0             |
|2012-01-01 11:06|0.0         |0.0             |
|2012-01-01 11:07|0.0         |0.0             |
|2012-01-01 11:08|0.0         |0.0             |
|2012-01-01 11:09|0.0         |0.0             |
|2012-01-01 11:10|0.0         |0.0             |
+----------------+------------+----------------+
only showing top 10 rows



In [17]:
from pyspark.sql.functions import hour, corr

# Extract the hour from the timestamp
correlated_df = correlated_df.withColumn("hour_of_day", hour(col("date")))

# Compute correlation between hour of the tweet and BTC price change
time_correlation = correlated_df.select(corr("hour_of_day", "price_change")).collect()[0][0]

print("Correlation between the hour of a tweet and BTC price change:", time_correlation)

Correlation between the hour of a tweet and BTC price change: -0.0004680098864745879
